In [7]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from dataset import NTUSkeletonDataset
from torch.utils.data import Dataset, DataLoader



In [8]:
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [9]:
# Root directory for dataset
dataroot = "processed_mutual_data"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Number of channels in the training images. For color images this is 3
nc = 2

# Size of z latent vector (i.e. size of generator input)
z = 100

# Number of training epochs
num_epochs = 20

# Learning rate for optimizers
lr = 0.0005

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [10]:
trainset = NTUSkeletonDataset(root_dir=dataroot,pinpoint=10)
trainloader = DataLoader(trainset, batch_size=1 ,shuffle=True, num_workers=4)

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")



In [11]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.concat_feat = None
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat,0.8))
            layers.append(nn.ReLU())
            return layers

        def after_blocks(in_feat, out_feat):
            layers = [nn.Dropout(0.5)]
            layers.append(nn.Linear(in_feat, out_feat))

        self.blocks_1 = nn.Sequential(
            *block(nz, 4096)
        )

        self.blocks_2 = nn.Sequential(
            *block(4096,4096)
            *block(4096,4096)
        )

        self.blocks_3 = nn.Sequential(
            *after_blocks(self.concat_feat,50)
        )

    def forward(self, z):
        b_1 = self.blocks_1(z)
        b_2 = self.blocks_2(b_1)
        concat_bs = torch.cat([b_1,b_2])
        self.concat_feat = concat_bs.shape[0]
        output_b = self.block_3(concat_bs)
        final_output = output_b.reshape(25,2)
        return final_output
        


In [ ]:
class Discriminator(nn.Module):class Discriminator(nn.Module):
    super(Discriminator, self).__init__()
    self.flatten_dim = None
    self.concat_feat = None

    def block(in_feat, out_feat):
        layers = [nn.Linear(in_feat, out_feat)]
        layers.append(nn.ReLU())

    def after_blocks(in_feat, out_feat):
        layers = [nn.Dropout(0.5)]
        layers.append(nn.Linear(in_feat, out_feat))

    self.blocks_1 = nn.Sequential(
        *block(self.flatten_dim, 4096)
    )
    self.blocks_2 = nn.Sequential(
        *block(4096, 4096),
        *block(4096, 4096),
    )
    self.blocks_3 = nn.Sequential(
        *after_blocks(self.concat_feat,1)
    )

    def forward(self, skeleton):
        skeleton_flatten = skeleton.reshape(50,)
        self.flatten_dim = skeleton_flatten.shape[0]
        b_1 = self.blocks_1(skeleton_flatten)
        b_2 = self.blocks_2(b_1)
        concat_bs = torch.cat([b_1,b_2])
        self.concat_feat = concat_bs.shape[0]
        validity = self.blocks_3(concat_bs)
        return validity
        
